<a href="https://colab.research.google.com/github/AmeliaSys/AdvML/blob/IMDB-Movie-Review-Text-Classification/Movie_Review_Text_Classification_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Amelia Sayes

ams2638

Adv ML Assignment #3

[Github Link](https://github.com/AmeliaSys/AdvML/blob/IMDB-Movie-Review-Text-Classification/Movie_Review_Text_Classification_Model.ipynb)



##File Setup

In [ ]:
#install aimodelshare library
! pip install aimodelshare==0.0.189

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 967.8/967.8 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.8/83.8 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 63.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.0/294.0 kB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 451.2/451.2 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.7/132.7 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.3/96.3 kB 8.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.0/147.0 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 61.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 M

In [ ]:
# Get competition data
from aimodelshare import download_data
download_data('public.ecr.aws/y2e2a1d6/sst2_competition_data-repository:latest') 

In [ ]:
# Set up X_train, X_test, and y_train_labels objects
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=Warning)

X_train=pd.read_csv("sst2_competition_data/X_train.csv", squeeze=True)
X_test=pd.read_csv("sst2_competition_data/X_test.csv", squeeze=True)

y_train_labels=pd.read_csv("sst2_competition_data/y_train_labels.csv", squeeze=True)

# ohe encode Y data
y_train = pd.get_dummies(y_train_labels)

X_train.head()

0    The Rock is destined to be the 21st Century 's...
1    The gorgeously elaborate continuation of `` Th...
2    Singer/composer Bryan Adams contributes a slew...
3                 Yet the act is still charming here .
4    Whether or not you 're enlightened by any of D...
Name: text, dtype: object

###2.   Preprocess data using keras tokenizer / Write and Save Preprocessor function


In [ ]:
# This preprocessor function makes use of the tf.keras tokenizer

from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences
import numpy as np

# Build vocabulary from training text data
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(X_train)

# preprocessor tokenizes words and makes sure all documents have the same length
def preprocessor(data, maxlen=40, max_words=10000):

    sequences = tokenizer.texts_to_sequences(data)

    word_index = tokenizer.word_index
    X = pad_sequences(sequences, maxlen=maxlen)

    return X

print(preprocessor(X_train).shape)
print(preprocessor(X_test).shape)

(6920, 40)
(1821, 40)


In [ ]:
import aimodelshare as ai
ai.export_preprocessor(preprocessor,"") 

Your preprocessor is now saved to 'preprocessor.zip'


In [ ]:
#Set credentials using modelshare.org username/password

from aimodelshare.aws import set_credentials
    
apiurl="https://rlxjxnoql9.execute-api.us-east-1.amazonaws.com/prod/m" #This is the unique rest api that powers this specific Playground

set_credentials(apiurl=apiurl)

AI Modelshare Username:··········
AI Modelshare Password:··········
AI Model Share login credentials set successfully.


##Code to Submit Model

In [ ]:
# Save keras model to local ONNX file
from aimodelshare.aimsonnx import model_to_onnx

onnx_model = model_to_onnx(model, framework='keras',
                          transfer_learning=False,
                          deep_learning=True)

with open("model.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

In [ ]:
#Instantiate competition
mycompetition= ai.Competition(apiurl)

In [ ]:
#Submit Model: 

#-- Generate predicted y values (Model 1)
#Note: Keras predict returns the predicted column index location for classification models
prediction_column_index=model.predict(preprocessor(X_test)).argmax(axis=1)

# extract correct prediction labels 
prediction_labels = [y_train.columns[i] for i in prediction_column_index]

# Submit Model 1 to Competition Leaderboard
mycompetition.submit_model(model_filepath = "model.onnx",
                                 preprocessor_filepath="preprocessor.zip",
                                 prediction_submission=prediction_labels)

57/57 [==============================] - 5s 54ms/step
Insert search tags to help users find your model (optional): 
Provide any useful notes about your model (optional): 

Your model has been submitted as model version 98

To submit code used to create this model or to view current leaderboard navigate to Model Playground: 

 https://www.modelshare.org/detail/model:2763


#1. Discuss the dataset in general terms and describe why building a predictive model using this data might be practically useful.  Who could benefit from a model like this? Explain.


This dataset consists of X data which are movie reviews and y data which is a label of the review either being positive or negative. The goal of this predictive model is to classify movie reviews correctly as either positive or negative. 

A predictive model able to classify sentiment of text is helpful in a number of ways. E.g. To calculate average sentiment across reviews for a movie. Platforms like IMDB and Rotten Tomatoes can use this information to produce a rating of movies. Not only is this helpful for the company, this is helpful to people who want to know how good a movie is.

In [ ]:
X_train

0       The Rock is destined to be the 21st Century 's...
1       The gorgeously elaborate continuation of `` Th...
2       Singer/composer Bryan Adams contributes a slew...
3                    Yet the act is still charming here .
4       Whether or not you 're enlightened by any of D...
                              ...                        
6915                                      A real snooze .
6916                                       No surprises .
6917    We 've seen the hippie-turned-yuppie plot befo...
6918    Her fans walked out muttering words like `` ho...
6919                                  In this case zero .
Name: text, Length: 6920, dtype: object

In [ ]:
y_train

,Negative,Positive
0,0,1
1,0,1
2,0,1
3,0,1
4,0,1
...,...,...
6915,1,0
6916,1,0
6917,0,1
6918,1,0


#2. Run at least three prediction models to try to predict the SST sentiment dataset well.


In [ ]:
max_len = 40

##2.a. Use Embedding layers and LSTM layers in at least one model


###Model #60

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Flatten

model2 = Sequential()
model2.add(Embedding(10000, 16, input_length=max_len))
model2.add(LSTM(100, return_sequences=True, dropout=0.2))
model2.add(LSTM(100, dropout=0.3))
model2.add(Flatten())
model2.add(Dense(2, activation='softmax'))

model2.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 40, 16)            160000    
                                                                 
 lstm_4 (LSTM)               (None, 40, 100)           46800     
                                                                 
 lstm_5 (LSTM)               (None, 100)               80400     
                                                                 
 flatten_2 (Flatten)         (None, 100)               0         
                                                                 
 dense_4 (Dense)             (None, 2)                 202       
                                                                 
Total params: 287,402
Trainable params: 287,402
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model2.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model2.fit(preprocessor(X_train), y_train,
                    epochs=5,
                    batch_size=32,
                    validation_split=0.2)

Epoch 1/5
173/173 [==============================] - 21s 104ms/step - loss: 0.6593 - acc: 0.6275 - val_loss: 0.7635 - val_acc: 0.4191
Epoch 2/5
173/173 [==============================] - 17s 97ms/step - loss: 0.4811 - acc: 0.7731 - val_loss: 0.6508 - val_acc: 0.6922
Epoch 3/5
173/173 [==============================] - 15s 90ms/step - loss: 0.3638 - acc: 0.8447 - val_loss: 0.5762 - val_acc: 0.7095
Epoch 4/5
173/173 [==============================] - 15s 89ms/step - loss: 0.2996 - acc: 0.8737 - val_loss: 0.6502 - val_acc: 0.7038
Epoch 5/5
173/173 [==============================] - 16s 95ms/step - loss: 0.2523 - acc: 0.8990 - val_loss: 0.5360 - val_acc: 0.7601


In [ ]:
data = mycompetition.get_leaderboard()
compar = data[["version", "accuracy", "f1_score", "precision", "recall", "model_type"]]
print(compar[compar["version"].isin([60])])

   version  accuracy  f1_score  precision    recall  model_type
6       60  0.792536  0.792094   0.795156  0.792587  Sequential


##2.b. Use Embedding layers and Conv1d layers in at least one model


### Model 61

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import SimpleRNN, LSTM,Embedding

model = Sequential()
model.add(layers.Embedding(10000, 8, input_length=max_len))
model.add(layers.Conv1D(50, 7, activation='relu')) 
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(2, activation='softmax'))

model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_10 (Embedding)    (None, 40, 8)             80000     
                                                                 
 conv1d_9 (Conv1D)           (None, 34, 50)            2850      
                                                                 
 global_max_pooling1d_3 (Glo  (None, 50)               0         
 balMaxPooling1D)                                                
                                                                 
 dense_8 (Dense)             (None, 2)                 102       
                                                                 
Total params: 82,952
Trainable params: 82,952
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(preprocessor(X_train), y_train,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.2)

Epoch 1/10
173/173 [==============================] - 2s 6ms/step - loss: 0.6652 - acc: 0.6149 - val_loss: 0.8895 - val_acc: 0.1488
Epoch 2/10
173/173 [==============================] - 1s 6ms/step - loss: 0.6280 - acc: 0.6239 - val_loss: 0.8109 - val_acc: 0.3266
Epoch 3/10
173/173 [==============================] - 1s 5ms/step - loss: 0.5188 - acc: 0.7520 - val_loss: 0.7587 - val_acc: 0.5499
Epoch 4/10
173/173 [==============================] - 1s 5ms/step - loss: 0.4034 - acc: 0.8331 - val_loss: 0.5764 - val_acc: 0.7298
Epoch 5/10
173/173 [==============================] - 1s 6ms/step - loss: 0.3286 - acc: 0.8680 - val_loss: 0.5973 - val_acc: 0.7153
Epoch 6/10
173/173 [==============================] - 1s 6ms/step - loss: 0.2804 - acc: 0.8864 - val_loss: 0.5877 - val_acc: 0.7298
Epoch 7/10
173/173 [==============================] - 1s 5ms/step - loss: 0.2423 - acc: 0.9005 - val_loss: 0.6672 - val_acc: 0.6879
Epoch 8/10
173/173 [==============================] - 1s 6ms/step - loss: 0.

In [ ]:
data = mycompetition.get_leaderboard()
compar = data[["version", "accuracy", "f1_score", "precision", "recall", "model_type"]]
print(compar[compar["version"].isin([60, 61])])

   version  accuracy  f1_score  precision    recall  model_type
6       60  0.792536  0.792094   0.795156  0.792587  Sequential
7       61  0.788145  0.786414   0.798021  0.788245  Sequential


##2.c. Use transfer learning with glove embeddings for at least one of these models


### Model #84

In [ ]:
# change max len to be 100 to be compatible with glove
def preprocessor(data, maxlen=50, max_words=10000):

    sequences = tokenizer.texts_to_sequences(data)

    word_index = tokenizer.word_index
    X = pad_sequences(sequences, maxlen=maxlen)

    return X

print(preprocessor(X_train).shape)
print(preprocessor(X_test).shape)

(6920, 50)
(1821, 50)


In [ ]:
! wget http://nlp.stanford.edu/data/wordvecs/glove.6B.zip

--2023-04-13 14:47:34--  http://nlp.stanford.edu/data/wordvecs/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/wordvecs/glove.6B.zip [following]
--2023-04-13 14:47:34--  https://nlp.stanford.edu/data/wordvecs/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/wordvecs/glove.6B.zip [following]
--2023-04-13 14:47:34--  https://downloads.cs.stanford.edu/nlp/data/wordvecs/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182753 (822M) [app

In [ ]:
! unzip glove.6B.zip 

Archive:  glove.6B.zip
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       
  inflating: glove.6B.50d.txt        


In [ ]:
# Extract embedding data for 100 feature embedding matrix
import os
glove_dir = os.getcwd()

embeddings_index = {}
f = open(os.path.join(glove_dir, 'glove.6B.50d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400001 word vectors.


In [ ]:
embedding_dim = 50 # change if you use txt files using larger number of features
max_words = 10000
word_index = tokenizer.word_index

embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if i < max_words:
        if embedding_vector is not None:
            # Words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector

In [ ]:
import tensorflow as tf
max_len = 50

model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(max_words, embedding_dim, input_length=max_len)) 
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(50, activation='relu'))
model.add(tf.keras.layers.Dense(50, activation='relu'))
model.add(tf.keras.layers.Dense(2, activation='softmax'))
model.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_12 (Embedding)    (None, 50, 50)            500000    
                                                                 
 flatten_12 (Flatten)        (None, 2500)              0         
                                                                 
 dense_28 (Dense)            (None, 50)                125050    
                                                                 
 dense_29 (Dense)            (None, 50)                2550      
                                                                 
 dense_30 (Dense)            (None, 2)                 102       
                                                                 
Total params: 627,702
Trainable params: 627,702
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False

In [ ]:
model.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_12 (Embedding)    (None, 50, 50)            500000    
                                                                 
 flatten_12 (Flatten)        (None, 2500)              0         
                                                                 
 dense_28 (Dense)            (None, 50)                125050    
                                                                 
 dense_29 (Dense)            (None, 50)                2550      
                                                                 
 dense_30 (Dense)            (None, 2)                 102       
                                                                 
Total params: 627,702
Trainable params: 127,702
Non-trainable params: 500,000
_________________________________________________________________


In [ ]:
model.compile(optimizer=tf.keras.optimizers.experimental.RMSprop(learning_rate = 0.0001), loss='binary_crossentropy', metrics=['acc'])

history = model.fit(preprocessor(X_train), y_train,
                    epochs=20,
                    batch_size=32,
                    validation_split=0.2)

Epoch 1/20
173/173 [==============================] - 3s 10ms/step - loss: 0.6689 - acc: 0.6109 - val_loss: 0.8645 - val_acc: 0.1510
Epoch 2/20
173/173 [==============================] - 2s 10ms/step - loss: 0.6356 - acc: 0.6201 - val_loss: 0.8439 - val_acc: 0.1929
Epoch 3/20
173/173 [==============================] - 2s 11ms/step - loss: 0.6045 - acc: 0.6530 - val_loss: 0.8519 - val_acc: 0.2818
Epoch 4/20
173/173 [==============================] - 1s 8ms/step - loss: 0.5755 - acc: 0.6895 - val_loss: 0.8204 - val_acc: 0.3996
Epoch 5/20
173/173 [==============================] - 1s 5ms/step - loss: 0.5471 - acc: 0.7188 - val_loss: 0.8010 - val_acc: 0.4783
Epoch 6/20
173/173 [==============================] - 1s 6ms/step - loss: 0.5206 - acc: 0.7444 - val_loss: 0.7489 - val_acc: 0.5621
Epoch 7/20
173/173 [==============================] - 1s 5ms/step - loss: 0.4968 - acc: 0.7619 - val_loss: 0.7064 - val_acc: 0.6199
Epoch 8/20
173/173 [==============================] - 1s 5ms/step - loss:

In [ ]:
data = mycompetition.get_leaderboard()
compar = data[["version", "accuracy", "f1_score", "precision", "recall", "model_type"]]
print(compar[compar["version"].isin([60, 61, 84])].sort_values("f1_score", ascending = False))

    version  accuracy  f1_score  precision    recall  model_type
11       60  0.792536  0.792094   0.795156  0.792587  Sequential
12       61  0.788145  0.786414   0.798021  0.788245  Sequential
56       84  0.678375  0.675876   0.684226  0.678473  Sequential


#3. Discuss which models performed better and point out relevant hyper-parameter values for successful models.
Submit your best three models to the leader board for the SST Model Share competition.

In [ ]:
data = mycompetition.get_leaderboard()
compar = data[["version", "accuracy", "f1_score", "precision", "recall", "model_type"]]
print(compar[compar["version"].isin([60, 61, 84])].sort_values("f1_score", ascending = False))

    version  accuracy  f1_score  precision    recall  model_type
11       60  0.792536  0.792094   0.795156  0.792587  Sequential
12       61  0.788145  0.786414   0.798021  0.788245  Sequential
56       84  0.678375  0.675876   0.684226  0.678473  Sequential


The highest performing model, #60 is my first model which consists of 1 embedding layer with 10,000 input dimensions and 16 output dimensions, and 2 LSTM layers, each with 100 neurons. The first LSTM layer has a dropout rate of 0.2 and the secon has a dropout rate of 0.3. The model is flattened before the dense layer.This layer used the provided preprocessing technique that limits reviews to 40 words only. After training for 5 epochs in batches of 32, it rendered a f1_score of 0.792.

This is only slightly higher than model #61, which uses 1 Embedding layer with 10000 input dimensions and 8 output dimensions and 1 Conv1D layer with 50 filters and a kernel size of 7. The Conv1D layer is activated by relu and is followed by Global MaxPooling 1D. This is a more simple model compared to #60. The f1_score on the test data is 0.786.

The weakest model was #84 which uses transfer learning with glove embeddings. Even though this is a more compelx model, the results are significantly weaker than the other models with an f1-score of only 0.676.

#4. Fit and submit up to three more models after learning from your team.


## 4.a. Model #85
LSTM and Embedding model with more layers to improve on Model #60

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Flatten
max_len = 40

model = Sequential()
model.add(Embedding(10000, 16, input_length=max_len))
model.add(LSTM(50, return_sequences=True, dropout=0.2))
model.add(LSTM(50, return_sequences=True, dropout=0.2))
model.add(LSTM(80, dropout=0.3))
model.add(Flatten())
model.add(Dense(2, activation='softmax'))

model.summary()

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_16 (Embedding)    (None, 40, 16)            160000    
                                                                 
 lstm_25 (LSTM)              (None, 40, 50)            13400     
                                                                 
 lstm_26 (LSTM)              (None, 40, 50)            20200     
                                                                 
 lstm_27 (LSTM)              (None, 80)                41920     
                                                                 
 flatten (Flatten)           (None, 80)                0         
                                                                 
 dense_15 (Dense)            (None, 2)                 162       
                                                                 
Total params: 235,682
Trainable params: 235,682
Non-t

In [ ]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

history = model.fit(preprocessor(X_train), y_train,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.2)

Epoch 1/10
173/173 [==============================] - 28s 117ms/step - loss: 0.6535 - acc: 0.6293 - val_loss: 0.7233 - val_acc: 0.4986
Epoch 2/10
173/173 [==============================] - 19s 112ms/step - loss: 0.4876 - acc: 0.7679 - val_loss: 0.5359 - val_acc: 0.7522
Epoch 3/10
173/173 [==============================] - 15s 84ms/step - loss: 0.3767 - acc: 0.8277 - val_loss: 0.7775 - val_acc: 0.6156
Epoch 4/10
173/173 [==============================] - 16s 91ms/step - loss: 0.3078 - acc: 0.8663 - val_loss: 0.5354 - val_acc: 0.7616
Epoch 5/10
173/173 [==============================] - 16s 93ms/step - loss: 0.2601 - acc: 0.8911 - val_loss: 0.4058 - val_acc: 0.8338
Epoch 6/10
173/173 [==============================] - 15s 87ms/step - loss: 0.2162 - acc: 0.9061 - val_loss: 0.6450 - val_acc: 0.7594
Epoch 7/10
173/173 [==============================] - 14s 84ms/step - loss: 0.1907 - acc: 0.9216 - val_loss: 0.5208 - val_acc: 0.8100
Epoch 8/10
173/173 [==============================] - 15s 87

In [ ]:
data = mycompetition.get_leaderboard()
compar = data[["version", "accuracy", "f1_score", "precision", "recall", "model_type"]]
print(compar[compar["version"].isin([60, 85])].sort_values("f1_score", ascending = False))

    version  accuracy  f1_score  precision    recall  model_type
3        85  0.814490  0.814199   0.816566  0.814534  Sequential
12       60  0.792536  0.792094   0.795156  0.792587  Sequential


Introducing more layers has improved f1 score from 0.79 to 0.81.

There are now 3 LSTM layers in the model, the first two with 50 neurons and dropout rate of 0.2 and the final layer with 80 and a dropout rate of 0.3. Model #60 in comparison had 2 LSTM layers with 100 neurons each, the first with a dropout rate of 0.2 and the second with a dropout rate of 0.3. This model is also trained for 10 epochs instead of the original 5.

##4.b. Model #86
Retrain Model #85 with max_len = 100

In [ ]:
# This preprocessor function makes use of the tf.keras tokenizer

from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences
import numpy as np

# Build vocabulary from training text data
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(X_train)

# preprocessor tokenizes words and makes sure all documents have the same length
def preprocessor(data, maxlen=100, max_words=10000):

    sequences = tokenizer.texts_to_sequences(data)

    word_index = tokenizer.word_index
    X = pad_sequences(sequences, maxlen=maxlen)

    return X

print(preprocessor(X_train).shape)
print(preprocessor(X_test).shape)

(6920, 100)
(1821, 100)


In [ ]:
import aimodelshare as ai
ai.export_preprocessor(preprocessor,"") 

Your preprocessor is now saved to 'preprocessor.zip'


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Flatten
max_len = 100

model = Sequential()
model.add(Embedding(10000, 16, input_length=max_len))
model.add(LSTM(50, return_sequences=True, dropout=0.2))
model.add(LSTM(50, return_sequences=True, dropout=0.2))
model.add(LSTM(80, dropout=0.3))
model.add(Flatten())
model.add(Dense(2, activation='softmax'))

model.summary()

Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_20 (Embedding)    (None, 100, 16)           160000    
                                                                 
 lstm_35 (LSTM)              (None, 100, 50)           13400     
                                                                 
 lstm_36 (LSTM)              (None, 100, 50)           20200     
                                                                 
 lstm_37 (LSTM)              (None, 80)                41920     
                                                                 
 flatten_2 (Flatten)         (None, 80)                0         
                                                                 
 dense_19 (Dense)            (None, 2)                 162       
                                                                 
Total params: 235,682
Trainable params: 235,682
Non-t

In [ ]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

history = model.fit(preprocessor(X_train), y_train,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.2)

Epoch 1/10
173/173 [==============================] - 56s 275ms/step - loss: 0.6495 - acc: 0.6339 - val_loss: 0.7335 - val_acc: 0.6062
Epoch 2/10
173/173 [==============================] - 50s 288ms/step - loss: 0.4858 - acc: 0.7711 - val_loss: 0.6673 - val_acc: 0.6431
Epoch 3/10
173/173 [==============================] - 48s 275ms/step - loss: 0.3638 - acc: 0.8387 - val_loss: 0.6045 - val_acc: 0.7247
Epoch 4/10
173/173 [==============================] - 49s 281ms/step - loss: 0.3377 - acc: 0.8696 - val_loss: 4.0732 - val_acc: 0.1488
Epoch 5/10
173/173 [==============================] - 49s 283ms/step - loss: 0.2696 - acc: 0.8941 - val_loss: 0.6206 - val_acc: 0.7312
Epoch 6/10
173/173 [==============================] - 47s 273ms/step - loss: 0.2242 - acc: 0.9099 - val_loss: 0.5288 - val_acc: 0.7601
Epoch 7/10
173/173 [==============================] - 50s 288ms/step - loss: 0.1888 - acc: 0.9230 - val_loss: 0.5673 - val_acc: 0.7782
Epoch 8/10
173/173 [==============================] - 4

In [ ]:
data = mycompetition.get_leaderboard()
compar = data[["version", "accuracy", "f1_score", "precision", "recall", "model_type"]]
print(compar[compar["version"].isin([60, 85, 86])].sort_values("f1_score", ascending = False))

    version  accuracy  f1_score  precision    recall  model_type
3        85  0.814490  0.814199   0.816566  0.814534  Sequential
7        86  0.803513  0.802136   0.812417  0.803605  Sequential
14       60  0.792536  0.792094   0.795156  0.792587  Sequential


Greater max_len performances is actually weaker than max_len 40 as default preprocessing and the same exact model architecture. 0.802 f1_score vs. 0.814. Still performs better than model #60.

##4.c. Model #97
Bidirectional LSTM

In [ ]:
from tensorflow.keras.layers import Embedding, Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Flatten
maxlen = 40

model = Sequential()
model.add(Embedding(10000, 16, input_length=maxlen))
model.add(Bidirectional(LSTM(50, return_sequences = True, dropout = 0.2)))
model.add(LSTM(50, dropout = 0.3))
model.add(Dense(2, activation='softmax'))

In [ ]:
import tensorflow as tf
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

history = model.fit(preprocessor(X_train), y_train,
                    epochs=5,
                    batch_size=100,
                    validation_split=0.2)

Epoch 1/5
56/56 [==============================] - 24s 257ms/step - loss: 0.6604 - acc: 0.6142 - val_loss: 1.8926 - val_acc: 0.1936
Epoch 2/5
56/56 [==============================] - 8s 151ms/step - loss: 0.5005 - acc: 0.7697 - val_loss: 0.4933 - val_acc: 0.8071
Epoch 3/5
56/56 [==============================] - 12s 220ms/step - loss: 0.3556 - acc: 0.8475 - val_loss: 0.6410 - val_acc: 0.6799
Epoch 4/5
56/56 [==============================] - 9s 169ms/step - loss: 0.2738 - acc: 0.8940 - val_loss: 0.5301 - val_acc: 0.7803
Epoch 5/5
56/56 [==============================] - 12s 209ms/step - loss: 0.2121 - acc: 0.9198 - val_loss: 0.4735 - val_acc: 0.7760


In [ ]:
data = mycompetition.get_leaderboard()
compar = data[["version", "accuracy", "f1_score", "precision", "recall", "model_type"]]
print(compar[compar["version"].isin([60, 85, 86, 97])].sort_values("f1_score", ascending = False))

    version  accuracy  f1_score  precision    recall  model_type
3        85  0.814490  0.814199   0.816566  0.814534  Sequential
4        97  0.811196  0.810978   0.812730  0.811235  Sequential
8        86  0.803513  0.802136   0.812417  0.803605  Sequential
16       60  0.792536  0.792094   0.795156  0.792587  Sequential


This bidirectional layer model is the second top performing model with a f1 score of 0.811, only just behind #85 at 0.814. 

This model has 1 embedding layer with 10000 inputs and 16 outputs, 1 bidirectional LSTM layer with 50 neurons, and dropout of 0.2, and a regular LSTM layer with 50 neurons and dropout of 0.3. The dropout layers were introduced to prevent overfitting. 

#5. Discuss results
Discuss which models you tried and which models performed better and point out relevant hyper-parameter values for successful models.

In [ ]:
data = mycompetition.get_leaderboard()
compar = data[["version", "accuracy", "f1_score", "precision", "recall", "model_type"]]
print(compar[compar["version"].isin([60, 85, 86, 97, 61, 84])].sort_values("f1_score", ascending = False))

    version  accuracy  f1_score  precision    recall  model_type
3        85  0.814490  0.814199   0.816566  0.814534  Sequential
4        97  0.811196  0.810978   0.812730  0.811235  Sequential
8        86  0.803513  0.802136   0.812417  0.803605  Sequential
16       60  0.792536  0.792094   0.795156  0.792587  Sequential
18       61  0.788145  0.786414   0.798021  0.788245  Sequential
69       84  0.678375  0.675876   0.684226  0.678473  Sequential


**Weaker Performing Models:** Of the 6 models I created, the three that performed the worst used glove embedding + transfer learning (#84), and embedding layers + conv1D layers (#61). Of my initial models, #60 was the best performer with an f1 score of 0.79. This model consists of 1 embedding layer with 10,000 input dimensions and 16 output dimensions, and 2 LSTM layers, each with 100 neurons. The first LSTM layer has a dropout rate of 0.2 and the secon has a dropout rate of 0.3. The model is flattened before the dense layer.This layer used the provided preprocessing technique that limits reviews to 40 words only. After training for 5 epochs in batches of 32, it rendered a f1_score of 0.792.

I then improved upon model #60 in part 4 and produced 3 new models:

**Best Performing Model: Model #85:** Model #85 is very similar to model #60 but more complex. This mdoel has 3 LSTM layers instead of 2 as in #60. The first two have 50 neurons and a dropout of 0.2 and the third layer has 80 neurons with dropout of 0.3. This model is also trained for 10 epochs instead of the original 5. This improved performance from 0.792 to 0.814.

Model #97 and #86 were also from part 4 of the assignment, and aimed to improve upon the performance of model #60. 

**Second Best Performing Model: Model #97:** This model rendered a f1-score of 0.813 which is very similar to the performance of #85 at 0.814. In this model I experimented with bidirectional LSTM layers. This model has 1 embedding layer with 10000 inputs and 16 outputs, 1 bidirectional LSTM layer with 50 neurons, and dropout of 0.2, and a regular LSTM layer with 50 neurons and dropout of 0.3. The dropout layers were introduced to prevent overfitting.  

In model #86 I experimented with different max lengths of the input text. Here, I changed the preprocessing of the text to a max length of 100 instead of 40. I used the same arhictecture as model #85 and found that this actually decreased performance. 

